In [1]:
import numpy as np
import os
import time
import datetime
import threading
import pandas as pd
import pickle
from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
from typing import Mapping
mp_pose = mp.solutions.pose

from audSample import *
from createAudio import *

pygame 2.1.2 (SDL 2.0.18, Python 3.9.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
audioFiles/15sec.wav exists
audioFiles/30sec.wav exists
audioFiles/45sec.wav exists
audioFiles/60sec.wav exists
audioFiles/75sec.wav exists
audioFiles/90sec.wav exists
audioFiles/105sec.wav exists
audioFiles/120sec.wav exists
audioFiles/135sec.wav exists
audioFiles/150sec.wav exists
audioFiles/165sec.wav exists
audioFiles/180sec.wav exists


In [ ]:
!pip install scikit-learn==1.0.1

In [3]:
classifier = pickle.load(open('yoga_model/latest_model1', 'rb'))

In [4]:
# to calculate angle in three points
def getAngles(item, pt1, pt2, pt3):
    a = item[pt1]
    b = item[pt2]
    c = item[pt3]
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = (np.arccos(cosine_angle))*(180/np.pi)
    return angle

In [3]:
def chk_vis(lis):
    cnt = 0
    for obj in lis:
        if (obj.visibility>0.5): cnt+=1
    if (cnt==33): return True
    else: return False 

In [5]:

from timeit import default_timer as timer

PosesAudioLoc = "audioFiles/poses/"
SecAudioLoc = "audioFiles/seconds/"


In [6]:
import json
colr = 255
with open("pose_angle.json", "r") as pose_angle:
    pose_data = json.load(pose_angle)
def chck_angle(ps_name, angle_dic):
    angle_col_dic = {}
    for d_angle in pose_data[ps_name]:
        mn, std = pose_data[ps_name][d_angle]
        rat = abs(mn-angle_dic[d_angle])/mn
        if rat>1: rat = 1
        angle_col_dic[d_angle] = (0, colr*(1-rat), colr*rat)
    print(angle_col_dic)
    return angle_col_dic
# print(pose_data)    

In [7]:
recently = []
steady_time = []
def start_vid(frame_rate):
    secCheck = 0
    vid = cv2.VideoCapture(0)
    # frame_rate = 30
    prev = time.time()
    threadChk = 0
    while(True):
        
        # Capture the video frame
        # by frame
        time_elapsed = time.time() - prev
        ret, frame = vid.read()
        # time_elapsed = time.time() - prev
        # ret, frame = self.video.read()
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            with mp_pose.Pose() as pose_tracker:
                result = pose_tracker.process(image=frame)
                # mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
                pose_landmarks = result.pose_landmarks
            output_frame = frame.copy()
            # print(pose_landmarks.landmark)
            pose_name = ""
            if pose_landmarks is not None:
                dict_obj = {}
                for i in range(0,33):
                    dict_obj[i] = mp_drawing.DrawingSpec(color=mp_drawing.WHITE_COLOR)
                pose_landmarka = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
                output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
                frame_height, frame_width = frame.shape[:2]
                pose_landmarka *= np.array([frame_width, frame_height, frame_width])
                # uncomment this when predicting
                item = pose_landmarka
                angle_dict = {
                    'left_shoulder': getAngles(item, 13, 11, 12),
                    'right_shoulder': getAngles(item, 24, 12, 14),
                    'left_elbow': getAngles(item, 15, 13, 11),
                    'right_elbow': getAngles(item, 12, 14, 16),
                    'left_leg_waist': getAngles(item, 11, 23, 25),
                    'right_leg_waist': getAngles(item, 12, 24, 26),
                    'left_knee': getAngles(item, 23, 25, 27),
                    'right_knee': getAngles(item, 24, 26, 28),
                }
                arr = np.array([angle_dict['left_shoulder'],
                angle_dict['right_shoulder'],
                angle_dict['left_elbow'],
                angle_dict['right_elbow'],
                angle_dict['left_leg_waist'],
                angle_dict['right_leg_waist'],
                angle_dict['left_knee'],
                angle_dict['right_knee']])
                res = classifier.predict([arr]) # classified pose name
                pose_name = res[0]
                angle_colr = chck_angle(pose_name, angle_dict)
                dict_obj[11] = mp_drawing.DrawingSpec(color=angle_colr["left_shoulder"])
                dict_obj[12] = mp_drawing.DrawingSpec(color=angle_colr["right_shoulder"])
                dict_obj[13] = mp_drawing.DrawingSpec(color=angle_colr["left_elbow"])
                dict_obj[14] = mp_drawing.DrawingSpec(color=angle_colr["right_elbow"])
                dict_obj[23] = mp_drawing.DrawingSpec(color=angle_colr["left_leg_waist"])
                dict_obj[24] = mp_drawing.DrawingSpec(color=angle_colr["right_leg_waist"])
                dict_obj[25] = mp_drawing.DrawingSpec(color=angle_colr["left_knee"])
                dict_obj[26] = mp_drawing.DrawingSpec(color=angle_colr["right_knee"])
                x: Mapping[int, mp_drawing.DrawingSpec] = dict_obj
                mp_drawing.draw_landmarks(
                    image=output_frame,
                    landmark_list=pose_landmarks,
                    connections=mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=  x)
                assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
                # if chk_vis(pose_landmarks.landmark):
                
            output_frame = cv2.putText(output_frame, pose_name, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
            # cv2.imshow('frame', output_frame)
            c_time = timer()
            
            if pose_name != "":
                steady_time_now = timer()
                steadyF = 0
                if not steady_time:
                    steady_time.append(steady_time_now)
                elif (steady_time_now - steady_time[0])>3:
                    steadyF = 1
                else:
                    # print(steady_time_now-steady_time[0])
                    steady_time.append(steady_time_now)
                    
                output_frame = cv2.putText(output_frame, str("{:.0f}".format(steady_time_now-steady_time[0]))+" s", (300,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                if steadyF == 1:
                    countSay = "{:.0f}".format(steady_time_now-steady_time[0])
                    if not recently: # classify and say
                        t1 = threading.Thread(target=start_playlist, args=[[ f"{PosesAudioLoc}{pose_name}.wav"]])
                        t1.start()
                        threadChk = 1
                        recently.append(c_time)

                    elif secCheck==0:
                        if int(countSay)%15 == 0:
                            print(countSay)
                            # basePerform = "seconds complete"
                            # textList = [f"{countSay} {basePerform}"]
                            # createAudio(textList, f"{audiosLoc}{countSay}sec.wav")
                            t1 = threading.Thread(target=start_playlist, args=[[ f"{SecAudioLoc}{countSay}sec.wav"]])
                            t1.start()
                            recently.pop()
                            recently.append(c_time)
                            secCheck = 1
                        else:
                            secCheck = 0
                    
                    if secCheck==1 and int(countSay)%15 != 0: secCheck=0
                    
                    
            else:
                steady_time.clear()
                recently.clear()
            cv2.imshow('frame', output_frame)
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

In [7]:
classifier.classes_

array(['adho mukha savanasana', 'ashtanga namaskar', 'aswa sanchalnasan',
       'bhujangasana', 'pranamasan', 'urdhva hastasana', 'uttanasana'],
      dtype=object)

In [29]:
classifier.metric_params()

2

In [8]:
start_vid(10)

d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'left_shoulder': (0, 253.02142750863973, 1.9785724913602656), 'right_shoulder': (0, 190.92014483643595, 64.07985516356403), 'left_elbow': (0, 14.40105611205409, 240.59894388794592), 'right_elbow': (0, 199.93068240257372, 55.06931759742627), 'left_leg_waist': (0, 162.20018357430664, 92.79981642569336), 'right_leg_waist': (0, 189.64664920410618, 65.35335079589385), 'left_knee': (0, 246.37910934443818, 8.620890655561828), 'right_knee': (0, 223.1880404744653, 31.81195952553467)}


d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'left_shoulder': (0, 46.14452261891925, 208.85547738108076), 'right_shoulder': (0, 8.694138658819005, 246.305861341181), 'left_elbow': (0, 243.0130466944457, 11.986953305554291), 'right_elbow': (0, 217.59556660529012, 37.404433394709905), 'left_leg_waist': (0, 254.21894673129088, 0.7810532687091113), 'right_leg_waist': (0, 254.68656724877522, 0.31343275122479297), 'left_knee': (0, 218.48170442769862, 36.51829557230139), 'right_knee': (0, 254.31070875044063, 0.6892912495593706)}


d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


...

{'left_shoulder': (0, 237.60105598230095, 17.39894401769903), 'right_shoulder': (0, 223.551676277929, 31.448323722071017), 'left_elbow': (0, 125.81605558474938, 129.18394441525064), 'right_elbow': (0, 245.65337411527616, 9.34662588472385), 'left_leg_waist': (0, 184.53692901774434, 70.46307098225563), 'right_leg_waist': (0, 221.5172802966409, 33.4827197033591), 'left_knee': (0, 223.2724342227578, 31.727565777242212), 'right_knee': (0, 185.82763725691268, 69.17236274308735)}


d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'left_shoulder': (0, 132.84398715821808, 122.1560128417819), 'right_shoulder': (0, 62.795957093113984, 192.20404290688603), 'left_elbow': (0, 182.5888660172157, 72.4111339827843), 'right_elbow': (0, 205.91707951067454, 49.08292048932545), 'left_leg_waist': (0, 131.4595068207026, 123.54049317929741), 'right_leg_waist': (0, 195.67010492002098, 59.32989507997903), 'left_knee': (0, 162.01867924372252, 92.98132075627748), 'right_knee': (0, 218.85508562417584, 36.14491437582414)}


d:\projects\YogaPoseDetect\yoga_jupy\env\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


{'left_shoulder': (0, 188.23303213433425, 66.76696786566572), 'right_shoulder': (0, 104.92266475269244, 150.07733524730756), 'left_elbow': (0, 228.4257185091297, 26.574281490870312), 'right_elbow': (0, 192.61733652902197, 62.38266347097803), 'left_leg_waist': (0, 179.67930195806207, 75.32069804193793), 'right_leg_waist': (0, 228.9205953647669, 26.079404635233082), 'left_knee': (0, 178.2996092386293, 76.70039076137073), 'right_knee': (0, 248.5900406509924, 6.409959349007606)}
